# RAG Reranking 실험 (2025-07-19)

## 실험 목적
`CrossEncoder 기반 reranking`을 적용하여 RAG 시스템의 검색 성능 개선

## 구성요소
- **임베딩**: OpenAI text-embedding-3-small
- **LLM**: Google Gemini 2.0 Flash  
- **Reranker**: BAAI/bge-reranker-v2-m3
- **DB**: ChromaDB
- **데이터**: HuggingFace 의료 QA 데이터셋 ("s1000secent/0705_rag_dataset"-20개 샘플)

## 평가 메트릭
- Hit Rate@k (k=1,3,5,10)
- MRR (Mean Reciprocal Rank)
- Semantic Similarity (임베딩 기반)

## 주요 결과
- 기본 평가: 데이터 매칭 이슈로 0.000
- **Semantic 분석**: 의미적으로 더 관련성 높은 문서가 상위로 배치
- **예시**: 부인과 증상 질문에서 유사도 0.354 → 0.523 (+47.7% 개선)

In [3]:
from dotenv import load_dotenv
import os

# .env 파일에서 환경 변수 로드
load_dotenv()

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_chroma import Chroma
from langchain.chains import HypotheticalDocumentEmbedder, LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [4]:
# 임베딩 모델 초기화 (OpenAI)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# LLM 초기화 (Google Gemini Flash)
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash",
)

In [5]:
# ChromaDB 영속 디렉토리 설정
PERSIST_DIRECTORY = r"C:\Users\Sese\AI_Study_Record\RAG_AGENT\rag_0719\chroma_db"
COLLECTION_NAME = "html_docs"

# ChromaDB 로드 및 Retriever 생성
db = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function=embeddings,
    collection_name=COLLECTION_NAME,
)
retriever = db.as_retriever()

In [6]:
from datasets import load_dataset
dataset = load_dataset("s1000secent/0705_rag_dataset", token=os.getenv("HUGGINGFACE_API_KEY"))
evaluation_data = dataset['0705']

In [7]:
# evaluation_data 구조 확인
print(f"데이터셋 크기: {len(evaluation_data)}")
print(f"첫 번째 샘플 키들: {evaluation_data[0].keys()}")
print(f"첫 번째 샘플:")
for key, value in evaluation_data[0].items():
    print(f"{key}: {value}")
    print("---")


데이터셋 크기: 5207
첫 번째 샘플 키들: dict_keys(['query', 'answer', 'relevant_docs_metadata'])
첫 번째 샘플:
query: 불명열 환자에서 무엇을 꼼꼼히 숙지해야 하는가?
---
answer: 불명열 진단적 접근 알고리즘
---
relevant_docs_metadata: [{'chunk_index': 0, 'chunk_metadata': {'Header 1': '#TITLE#'}, 'id': '63138f2a-6cd4-41e0-acb7-b4709755763f', 'source': 'theory_texts\\1636_3826_발열, 불명열.html', 'total_chunks': 3}]
---


In [24]:
# Reranker 설정
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
import numpy as np
from typing import List, Tuple
from sentence_transformers import CrossEncoder

# CrossEncoder 객체 사용
reranker = CrossEncoder("BAAI/bge-reranker-v2-m3")
def rerank_documents(query: str, documents: List[str], top_k: int = 5) -> List[Tuple[str, float]]:
    pairs = [(query, doc) for doc in documents]
    scores = reranker.predict(pairs)  # ✅ 이제 정상 작동

    doc_scores = list(zip(documents, map(float, scores)))
    doc_scores.sort(key=lambda x: x[1], reverse=True)
    return doc_scores[:top_k]


In [28]:
import random
import numpy as np
from tqdm import tqdm
from typing import List, Tuple

def calculate_hit_rate(relevant_docs: List[str], retrieved_docs: List[str], k: int = 5) -> float:
    """Hit Rate@k 계산"""
    retrieved_top_k = retrieved_docs[:k]
    hits = sum(1 for doc in relevant_docs if doc in retrieved_top_k)
    return min(hits / len(relevant_docs), 1.0) if relevant_docs else 0.0

def calculate_mrr(relevant_docs: List[str], retrieved_docs: List[str]) -> float:
    """Mean Reciprocal Rank 계산"""
    for i, doc in enumerate(retrieved_docs):
        if doc in relevant_docs:
            return 1.0 / (i + 1)
    return 0.0

def evaluate_retrieval_with_reranking(
    eval_samples: List[dict], 
    retriever, 
    reranker_func,
    k_values: List[int] = [1, 3, 5, 10]
) -> dict:
    """
    Retrieval + Reranking 평가
    """
    results = {f'hit_rate@{k}': [] for k in k_values}
    for k in k_values:
        results[f'hit_rate@{k}_original'] = []
    results['mrr'] = []
    results['mrr_original'] = []

    for sample in tqdm(eval_samples, desc="평가 중"):
        query = sample['query']
        ground_truth = sample['answer']
        
        # 기본 retrieval
        retrieved_docs = retriever.get_relevant_documents(query)
        original_docs = [doc.page_content for doc in retrieved_docs]
        
        # 관련 문서 정의
        relevant_docs = ground_truth if isinstance(ground_truth, list) else [ground_truth]

        # Original 평가
        for k in k_values:
            hit_rate_orig = calculate_hit_rate(relevant_docs, original_docs, k)
            results[f'hit_rate@{k}_original'].append(hit_rate_orig)

        mrr_orig = calculate_mrr(relevant_docs, original_docs)
        results['mrr_original'].append(mrr_orig)

        # Reranking
        if original_docs:
            reranked_results = reranker_func(query, original_docs, top_k=max(k_values))
            reranked_docs = [doc for doc, _ in reranked_results]
        else:
            reranked_docs = []

        for k in k_values:
            hit_rate = calculate_hit_rate(relevant_docs, reranked_docs, k)
            results[f'hit_rate@{k}'].append(hit_rate)

        mrr = calculate_mrr(relevant_docs, reranked_docs)
        results['mrr'].append(mrr)

    # 평균 결과 정리
    final_results = {}
    for k in k_values:
        final_results[f'hit_rate@{k}'] = np.mean(results[f'hit_rate@{k}'])
        final_results[f'hit_rate@{k}_original'] = np.mean(results[f'hit_rate@{k}_original'])
    final_results['mrr'] = np.mean(results['mrr'])
    final_results['mrr_original'] = np.mean(results['mrr_original'])

    return final_results

# 평가 샘플 준비
random.seed(42)
eval_samples = random.sample(list(evaluation_data), 20)
print(f"평가할 샘플 수: {len(eval_samples)}")
print(f"첫 번째 샘플:")
print(f"Query: {eval_samples[0]['query']}")
print(f"Answer: {eval_samples[0]['answer']}...")


평가할 샘플 수: 20
첫 번째 샘플:
Query: 부인과 환자의 주호소 중 가장 흔한 증상 중 하나는 무엇인가요?
Answer: 비정상 자궁출혈...


In [29]:
# 더 정교한 평가를 위한 semantic similarity 기반 함수
from sklearn.metrics.pairwise import cosine_similarity

def calculate_semantic_hit_rate(ground_truth: str, retrieved_docs: List[str], embeddings_model, k: int = 5, threshold: float = 0.7) -> float:
    """Semantic similarity 기반 Hit Rate@k 계산"""
    if not retrieved_docs or not ground_truth:
        return 0.0
    
    # Ground truth와 retrieved documents의 임베딩 생성
    gt_embedding = embeddings_model.embed_query(ground_truth)
    doc_embeddings = embeddings_model.embed_documents(retrieved_docs[:k])
    
    # Cosine similarity 계산
    similarities = cosine_similarity([gt_embedding], doc_embeddings)[0]
    
    # Threshold 이상의 유사도를 가진 문서가 있으면 hit
    return 1.0 if max(similarities) >= threshold else 0.0

def calculate_semantic_mrr(ground_truth: str, retrieved_docs: List[str], embeddings_model, threshold: float = 0.7) -> float:
    """Semantic similarity 기반 MRR 계산"""
    if not retrieved_docs or not ground_truth:
        return 0.0
    
    # Ground truth와 retrieved documents의 임베딩 생성
    gt_embedding = embeddings_model.embed_query(ground_truth)
    doc_embeddings = embeddings_model.embed_documents(retrieved_docs)
    
    # Cosine similarity 계산
    similarities = cosine_similarity([gt_embedding], doc_embeddings)[0]
    
    # Threshold 이상의 첫 번째 문서의 순위 찾기
    for i, sim in enumerate(similarities):
        if sim >= threshold:
            return 1.0 / (i + 1)
    
    return 0.0

def evaluate_with_semantic_similarity(
    eval_samples: List[dict], 
    retriever, 
    reranker_func,
    embeddings_model,
    k_values: List[int] = [1, 3, 5, 10],
    threshold: float = 0.7
) -> dict:
    """Semantic similarity 기반 평가"""
    results = {f'semantic_hit_rate@{k}': [] for k in k_values}
    results['semantic_mrr'] = []
    results[f'semantic_hit_rate@{k}_original'] = {k: [] for k in k_values}
    results['semantic_mrr_original'] = []
    
    for sample in tqdm(eval_samples, desc="Semantic 평가 중"):
        query = sample['query']  # 'question' -> 'query'로 수정
        ground_truth = sample['answer']
        
        # 1. 기본 retrieval
        retrieved_docs = retriever.get_relevant_documents(query)
        original_docs = [doc.page_content for doc in retrieved_docs]
        
        # 2. Reranking
        if len(original_docs) > 0:
            reranked_results = reranker_func(query, original_docs, top_k=10)
            reranked_docs = [doc for doc, score in reranked_results]
        else:
            reranked_docs = []
        
        # 3. Semantic similarity 기반 평가
        # Original retrieval 평가
        for k in k_values:
            hit_rate_orig = calculate_semantic_hit_rate(ground_truth, original_docs, embeddings_model, k, threshold)
            results[f'semantic_hit_rate@{k}_original'][k].append(hit_rate_orig)
        
        mrr_orig = calculate_semantic_mrr(ground_truth, original_docs, embeddings_model, threshold)
        results['semantic_mrr_original'].append(mrr_orig)
        
        # Reranked retrieval 평가
        for k in k_values:
            hit_rate = calculate_semantic_hit_rate(ground_truth, reranked_docs, embeddings_model, k, threshold)
            results[f'semantic_hit_rate@{k}'].append(hit_rate)
        
        mrr = calculate_semantic_mrr(ground_truth, reranked_docs, embeddings_model, threshold)
        results['semantic_mrr'].append(mrr)
    
    # 평균 계산
    final_results = {}
    for k in k_values:
        final_results[f'semantic_hit_rate@{k}'] = np.mean(results[f'semantic_hit_rate@{k}'])
        final_results[f'semantic_hit_rate@{k}_original'] = np.mean(results[f'semantic_hit_rate@{k}_original'][k])
    
    final_results['semantic_mrr'] = np.mean(results['semantic_mrr'])
    final_results['semantic_mrr_original'] = np.mean(results['semantic_mrr_original'])
    
    return final_results


In [30]:
# 실제 평가 실행
print("=== Reranker 평가 시작 ===")
print(f"평가 샘플 수: {len(eval_samples)}")
print(f"첫 번째 샘플 예시:")
print(f"Question: {eval_samples[0]['query']}")  # 'question' -> 'query'로 수정
print(f"Answer: {eval_samples[0]['answer']}...")
print()
results = evaluate_retrieval_with_reranking(
    eval_samples=eval_samples,
    retriever=retriever,
    reranker_func=rerank_documents,
    k_values=[1, 3, 5, 10]
)


=== Reranker 평가 시작 ===
평가 샘플 수: 20
첫 번째 샘플 예시:
Question: 부인과 환자의 주호소 중 가장 흔한 증상 중 하나는 무엇인가요?
Answer: 비정상 자궁출혈...



평가 중: 100%|██████████| 20/20 [04:17<00:00, 12.89s/it]


In [32]:
print("=== 평가 결과 ===")
print("\n📊 Hit Rate@k 비교 (Reranking 전 vs 후)")
print("-" * 50)

k_values = [1, 3, 5, 10]
for k in k_values:
    original = results[f'hit_rate@{k}_original']
    reranked = results[f'hit_rate@{k}']
    improvement = ((reranked - original) / original * 100) if original > 0 else 0

    print(f"Hit Rate@{k}")
    print(f"  원본:     {original:.3f}")
    print(f"  Reranked: {reranked:.3f}")
    print(f"  개선:     {improvement:+.1f}%")
    print()

print("📈 MRR (Mean Reciprocal Rank) 비교")
print("-" * 30)
original_mrr = results['mrr_original']
reranked_mrr = results['mrr']
mrr_improvement = ((reranked_mrr - original_mrr) / original_mrr * 100) if original_mrr > 0 else 0

print(f"원본 MRR:     {original_mrr:.3f}")
print(f"Reranked MRR: {reranked_mrr:.3f}")
print(f"MRR 개선:     {mrr_improvement:+.1f}%")

# 전체 결과 요약
print("\n🎯 종합 분석")
print("-" * 20)
if reranked_mrr > original_mrr:
    print("✅ Reranking이 전반적으로 성능을 향상시켰습니다!")
else:
    print("❌ Reranking이 성능을 저하시켰습니다.")

# 가장 큰 개선을 보인 k 값 찾기
best_k = None
best_improvement = -float('inf')

for k in k_values:
    original = results[f'hit_rate@{k}_original']
    reranked = results[f'hit_rate@{k}']
    improvement = reranked - original

    if improvement > best_improvement:
        best_improvement = improvement
        best_k = k

print(f"가장 큰 개선을 보인 k 값: {best_k} (개선: +{best_improvement:.3f})")


=== 평가 결과 ===

📊 Hit Rate@k 비교 (Reranking 전 vs 후)
--------------------------------------------------
Hit Rate@1
  원본:     0.000
  Reranked: 0.000
  개선:     +0.0%

Hit Rate@3
  원본:     0.000
  Reranked: 0.000
  개선:     +0.0%

Hit Rate@5
  원본:     0.000
  Reranked: 0.000
  개선:     +0.0%

Hit Rate@10
  원본:     0.000
  Reranked: 0.000
  개선:     +0.0%

📈 MRR (Mean Reciprocal Rank) 비교
------------------------------
원본 MRR:     0.000
Reranked MRR: 0.000
MRR 개선:     +0.0%

🎯 종합 분석
--------------------
❌ Reranking이 성능을 저하시켰습니다.
가장 큰 개선을 보인 k 값: 1 (개선: +0.000)


In [34]:
# 구체적인 예시 분석 (처음 3개 샘플)
print("\n🔍 구체적인 예시 분석")
print("=" * 50)

for i, sample in enumerate(eval_samples[:3]):
    print(f"\n📋 예시 {i+1}")
    print(f"Question: {sample['query']}")  # 'question' -> 'query'로 수정
    print(f"Ground Truth Answer: {sample['answer'][:150]}...")
    print()
    
    # Retrieval 수행
    query = sample['query']  # 'question' -> 'query'로 수정
    retrieved_docs = retriever.get_relevant_documents(query)
    original_docs = [doc.page_content for doc in retrieved_docs]
    
    # Reranking 수행
    if len(original_docs) > 0:
        reranked_results = rerank_documents(query, original_docs, top_k=5)
        reranked_docs = [doc for doc, score in reranked_results]
        rerank_scores = [score for doc, score in reranked_results]
    else:
        reranked_docs = []
        rerank_scores = []
    
    # 원본 결과 (상위 3개)
    print("📄 원본 Retrieval 결과 (상위 3개):")
    for j, doc in enumerate(original_docs[:3]):
        print(f"  {j+1}. {doc[:100]}...")
    
    print()
    
    # Reranked 결과 (상위 3개)
    print("🎯 Reranked 결과 (상위 3개):")
    for j, (doc, score) in enumerate(zip(reranked_docs[:3], rerank_scores[:3])):
        print(f"  {j+1}. (점수: {score:.3f}) {doc}...")
    
    # Semantic similarity 계산
    if original_docs:
        gt_embedding = embeddings.embed_query(sample['answer'])
        
        # 원본 상위 문서와의 유사도
        top_orig_embedding = embeddings.embed_documents([original_docs[0]])[0]
        orig_similarity = cosine_similarity([gt_embedding], [top_orig_embedding])[0][0]
        
        # Reranked 상위 문서와의 유사도 (있는 경우)
        if reranked_docs:
            top_reranked_embedding = embeddings.embed_documents([reranked_docs[0]])[0]
            reranked_similarity = cosine_similarity([gt_embedding], [top_reranked_embedding])[0][0]
        else:
            reranked_similarity = 0
        
        print(f"\n💯 Semantic Similarity with Ground Truth:")
        print(f"  원본 1위 문서:    {orig_similarity:.3f}")
        print(f"  Reranked 1위 문서: {reranked_similarity:.3f}")
        print(f"  개선:             {reranked_similarity - orig_similarity:+.3f}")
    
    print("\n" + "-" * 80)



🔍 구체적인 예시 분석

📋 예시 1
Question: 부인과 환자의 주호소 중 가장 흔한 증상 중 하나는 무엇인가요?
Ground Truth Answer: 비정상 자궁출혈...

📄 원본 Retrieval 결과 (상위 3개):
  1. 제목: 심부전 원인, 증상, 진단

4. 임상양상 
 심부전에 의한 증상은 크게 congestion(pulmonary, systemic)에 의한 증상(호흡곤란, 부종)과 reduc...
  2. 제목: 깊은정맥혈전증

2. 임상양상 
 1) 주호소:  사지 통증 ,   부종   (참고:  부종 감별 ) 
 (1) 주로 일측성으로 발생 
 (2) 압통, 오목부종 (pitti...
  3. 제목: 비정상 자궁출혈

: Abnormal uterine bleeding, AUB 
 
 
 부인과 환자의 주호소 중 가장 흔한 증상 중 하나로, 수많은 질환이 질출혈의 형태로 ...

🎯 Reranked 결과 (상위 3개):
  1. (점수: 0.998) 제목: 비정상 자궁출혈

: Abnormal uterine bleeding, AUB 
 
 
 부인과 환자의 주호소 중 가장 흔한 증상 중 하나로, 수많은 질환이 질출혈의 형태로 나타날 수 있다. 본 단원은 각 연령대별로 출혈의 다양한 원인에 대해 총론적인 내용을 담고 있으며, 각 질환별로 자세한 임상양상, 진단, 치료는 개별 단원에서 다루고 있다....
  2. (점수: 0.033) 제목: 환기저하

2. 임상양상 
 대개 비특이적이며 기저질환마다 조금씩 다르다. 
 
 1) 주호소:  호흡곤란 
 (1) 운동 시 호흡곤란으로 시작해 일상생활에 지장을 주는 정도까지 발전 
 (2) Orthopnea가 흔함 (누우면 복부 장기들이 위쪽으로 올라오기 때문) 
 
 2) 기타 증상 및 징후 
 (1) 수면의 질 하락, 주간졸림증(hypersomnolence), 과잉수면 
 (2) 두통, 불안 등...
  3. (점수: 0.016) 제목: 심부전 원인, 증상, 진단

4. 임상양상 
 심부전에